This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url='https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key=6FJQPxACHppVUvpUUmNm'
response=requests.get(url)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
dt=response.json()

In [5]:
type(dt)

dict

In [6]:
#first level
dt.keys()

dict_keys(['dataset'])

In [7]:
#second level
dt['dataset'].keys()

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])

In [8]:
#third level
dt['dataset']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

# Quandl API Exercise

### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [9]:
url17='https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?"+ "&start_date=2017-01-01&end_date=2017-12-31&api_key=6FJQPxACHppVUvpUUmNm'
response17=requests.get(url17)

### 2. Conver the returned JSON object into a Python dictionary

In [10]:
dt_17=response17.json()
type(dt_17)

dict

### 3.Calculate what the highest and lowest opening prices were for the stock in this period.

In [11]:
# get the first dataset
dt_17['dataset']['data'][0]

['2017-12-29',
 51.76,
 51.94,
 51.45,
 51.76,
 None,
 34640.0,
 1792304.0,
 None,
 None,
 None]

In [12]:
#check index on open column
dt_17['dataset']['column_names'].index('Open')

1

In [13]:
#extract the open prices into a list
open_prices=[]

data_17=dt_17['dataset']['data']
for price in data_17:
    if price[1] != None:
        open_prices.append(price[1])
    

In [14]:
print('The highest opening price is:', max(open_prices))

The highest opening price is: 53.11


In [15]:
print('The lowest opening price is:', min(open_prices))

The lowest opening price is: 34.0


### 4.What was the largest change in any one day (based on High and Low price)?

In [16]:
dt_17['dataset']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [17]:
diff=[]

for i in data_17:
    if (i[2] != None) & (i[3] != None):
        diff.append(i[2]-i[3])

In [18]:
print('The largest price change in any one day is:', max(diff))

The largest price change in any one day is: 2.8100000000000023


### 5.What was the largest change between any two days (based on Closing Price)?

In [19]:
closing=[]

for i in data_17:
    if i[4] != None:
        closing.append(i[4])

In [20]:
import numpy as np

In [21]:
#transform into array for calculation
next_closing=np.array(closing[1:])
pre_closing=np.array(closing[:len(closing)-1])

In [22]:
changes=next_closing-pre_closing
print('The largest change between any two days is:', max(abs(changes)))

The largest change between any two days is: 2.559999999999995


### 6.What was the average daily trading volume during this year?

In [23]:
#extract trading volume data
volume=[]
for i in data_17:
    volume.append(i[6])

In [24]:
#sum up the trading volumn for this year
sum_vol=0
for i in range(len(volume)):
    sum_vol=sum_vol+volume[i]

In [25]:
avg_vol= sum_vol/len(volume)

In [26]:
print('The average daily trading volume is:', avg_vol)

The average daily trading volume is: 89124.33725490196


### 7.What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [27]:
def find_median(val):
    sorted_values=sorted(volume)
    
    # check the length is old
    if len(val) % 2 == 1:
        return sorted_values[len(val)//2]
    
    # when the length is even
    return sum(sorted_values[len(val)//2 - 1 : len(val)//2 + 1])/2
        

In [28]:
print('The median trading volumn during this year is:', find_median(volume))

The median trading volumn during this year is: 76286.0
